In [1]:
import pandas as pd
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from skimage.transform import rescale, resize, downscale_local_mean
from torchvision import datasets, transforms
from PIL import Image
from tqdm import tqdm
import h5py
from transformers import *

In [2]:
#csv_read_in = pd.read_csv('./export_dataframe.csv', header=None)

In [3]:
#data_files = list(csv_read_in[0])

In [4]:
normalize = transforms.Normalize(
		mean=[0.485, 0.456, 0.406],
		std= [0.229, 0.224, 0.225])

transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((256,256)),
		transforms.ToTensor(),
		normalize,
		])

In [5]:
class CAD120Dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_read = pd.read_csv(csv_file, header=None)
        self.files = list(self.csv_read[0])
        self.transform = transform
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.files[idx]
        
        image = io.imread(img_name)

       
        if self.transform:
            image = self.transform(image)
            
        
        caption = self.csv_read[1][idx]

        sample = {'image':  image, 'caption': caption}

            
        return sample
        

In [6]:
cad120 = CAD120Dataset('./caption.csv', transform)


In [7]:
sample = cad120[501]

In [8]:
res50_model = models.resnet18(pretrained=True)
res50_model.cuda()
res50_model.eval()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/nikepupu/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 9.63MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
res50_conv = nn.Sequential(*list(res50_model.children())[:-2])
for p in res50_conv.parameters():
    p.requires_grad = False
res50_conv.cuda()
res50_conv.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [12]:
sample['caption']

' woman open microwave'

In [13]:
l = cad120.__len__()
#features = np.zeros((l, 2048,8,8), dtype=np.float32)
features = []
caption = []
model_class, tokenizer_class, pretrained_weights = (BertModel,BertTokenizer,'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
embedding_model = model_class.from_pretrained(pretrained_weights)
cap = ""
block = []
for i in tqdm(range(l)):
    sample = cad120[i]
   
    
    if cap != sample['caption']:
        if len(block) != 0:
            features.append(np.array(block))
            block = []
        input_ids = torch.tensor([tokenizer.encode(sample['caption'], add_special_tokens=True)])
        with torch.no_grad():
            res = embedding_model(input_ids)[0]
            
        caption.append(res)
            
    cap = sample['caption']
    img =  sample['image'].unsqueeze(0).cuda()
    a = res50_conv(img)
    a = a.squeeze(0)
    a = a.cpu().detach().numpy()
    a = np.asarray(a)
    block.append(a)

features.append(np.array(block))
features = np.array(features)
   
    
    

 14%|█▍        | 1696/12046 [00:33<03:05, 55.78it/s]

KeyboardInterrupt: 

In [ ]:
import pickle

In [ ]:
with open("caption.bin", "wb") as fp:   #Pickling
    pickle.dump(caption, fp)

In [ ]:
with open("features.bin", "wb") as fp:   #Pickling
    pickle.dump(features, fp)

In [ ]:
npfeatures = np.array(features)

In [ ]:
h5f = h5py.File('cad120_features.h5', 'w')
h5f.create_dataset('cad120_features', data=npfeatures)